In [1]:
import os, glob, numpy as np, nibabel as nib, matplotlib.pylab as plt
from sklearn.model_selection import train_test_split

In [2]:
master_path = r'./BraTS/'
folders = [folder for folder in os.listdir(os.path.join(master_path, 'BraTS2021_Training_Data')) if folder != '.DS_Store']

In [ ]:
if not os.path.exists(os.path.join(master_path, 'BraTS2021_Training_Data_2D')): # BraTS2021_Training_Data
    os.makedirs(os.path.join(master_path, 'BraTS2021_Training_Data_2D'))
    for img_type in ['flair', 'seg', 't1', 't1ce', 't2']:
        os.makedirs(os.path.join(master_path, 'BraTS2021_Training_Data_2D', img_type))

def convert(folders, org_folder = 'BraTS2021_Training_Data'):
    for counter, folder in enumerate(folders):
        for img_type in ['flair', 'seg', 't1', 't1ce', 't2']:
            img_path = os.path.join(master_path, org_folder, folder, folder + '_' + img_type + '.nii.gz')
            img = nib.load(img_path).get_fdata()
            for i in range(img.shape[-1]):
                img_slice = img[:,:,i]
                plt.imsave(os.path.join(master_path, 'BraTS2021_Training_Data_2D', img_type, folder + img_type + '_' + str(i+1) + '.png'), img_slice)
        print(f'Finished {counter+1}/{len(folders)}.', end="\r")


In [ ]:
convert(folders)

# Train Val Test Split

In [3]:
folders_train_tmp, folders_test = train_test_split(folders, test_size=0.1, random_state=42)
folders_train, folders_val = train_test_split(folders_train_tmp, test_size=0.22, random_state=42)

del(folders_train_tmp)

In [4]:
# We want the file structure to be

# Main
# |-- Train
#     |-- flair_001.png
#     |-- flair_002.png
#     |-- .json
# |-- Val
#     |-- flair_001.png
#     |-- flair_002.png
#     |-- .json
# |-- Test
#     |-- flair_001.png
#     |-- flair_002.png
#     |-- .json

In [5]:
if not os.path.exists(os.path.join(master_path, 'BraTS2021_Coco')): # BraTS2021_Training_Data
    os.makedirs(os.path.join(master_path, 'BraTS2021_Coco'))

datasets = ['Train', 'Val', 'Test']

def create_folder_structure(dataset, org_folder = 'BraTS2021_Training_Data'):
    dataset_ind = eval('folders_' + dataset.lower())
    for counter, folder in enumerate(dataset_ind):
        path_img = os.path.join(master_path, org_folder, folder, folder + '_flair.nii.gz')
        path_label = os.path.join(master_path, org_folder, folder, folder + '_seg.nii.gz')
        img = nib.load(path_img).get_fdata()
        label = nib.load(path_label).get_fdata()

        if not os.path.exists(os.path.join(master_path, 'BraTS2021_Coco', dataset)):
            os.makedirs(os.path.join(master_path, 'BraTS2021_Coco', dataset))
        # break images down
        for i in range(img.shape[-1]):
            img_slice = img[:,:,i]
            label_slice = label[:,:,i]

            # Separate labels
            label_slice1 = np.where(label_slice == 1, 1, 0)
            label_slice2 = np.where(label_slice == 2, 1, 0)
            label_slice4 = np.where(label_slice == 4, 1, 0)

            # Save images using matplotlib
            plt.imsave(os.path.join(master_path, 'BraTS2021_Coco', dataset, folder + '_flair' + '_' + str(i+1) + '.png'), img_slice)
            plt.imsave(os.path.join(master_path, 'BraTS2021_Coco', dataset, folder + '_seg' + '_level_1_' + str(i+1) + '.png'), label_slice1, cmap="gray")
            plt.imsave(os.path.join(master_path, 'BraTS2021_Coco', dataset, folder + '_seg' + '_level_2_' + str(i+1) + '.png'), label_slice2, cmap="gray")
            plt.imsave(os.path.join(master_path, 'BraTS2021_Coco', dataset, folder + '_seg' + '_level_4_' + str(i+1) + '.png'), label_slice4, cmap="gray")

            # Save images using cv2
            # np.save(os.path.join(master_path + '_array', folder, folder + '_flair' + '_' + str(i+1)), img_slice)
            # np.save(os.path.join(master_path + '_array', folder, folder + '_seg' + '_' + str(i+1)), label_slice)
        print(f'Finished {counter+1}/{len(dataset_ind)}: {folder} for {dataset} set.', end="\r")
    print('Finished all folders for ' + dataset + ' set. \n')


In [6]:
for dataset in datasets:
    create_folder_structure(dataset)

Finished all folders for Train set. r Train set.

Finished all folders for Val set. for Val set.

Finished all folders for Test set. or Test set.



In [7]:
# To load an image
# np.load('/tmp/123.npy')

In [11]:
for dataset in datasets:
    master_path = f"./BraTS/BraTS2021_Coco/{dataset}/"
    for level in ['level_1', 'level_2', 'level_4']:
        # find all level_1 files in Train etc
        mask_files = glob.glob(master_path + f'*{level}*.png')
        # Create file
        if not os.path.exists(os.path.join(master_path, level)):
            os.makedirs(os.path.join(master_path, level))
        # move all mask_files to it
        for mask_file in mask_files:
            os.rename(mask_file, os.path.join(master_path, level, os.path.basename(mask_file)))